<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240201-%E5%BE%97%E5%88%B02470%E6%A0%B7%E6%9C%AC%E7%9A%842022NDVI%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97(3.21-8.23).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [2]:
import os
import pandas as pd
import math

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/randomsample_2022_1000")

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

2646


In [5]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')


# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [6]:
# 5天序列NDVI
interval = 5
increment = 'day'
start = '2022-03-21'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-08-23').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDVI(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('NDVI'))

fitteds2 = s2makeup.map(add_fittedNDVI)

HANTS = fitteds2.select('NDVI')

In [7]:
# NDVI时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI', '25_NDVI', '26_NDVI', '27_NDVI', '28_NDVI', '29_NDVI', '30_NDVI', '31_NDVI']


In [8]:
image_agrgt_clip = image_agrgt.clip(roi)

In [9]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id'],scale=10,tileScale=8)

In [10]:
cols = ['id', '0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI', '25_NDVI', '26_NDVI', '27_NDVI', '28_NDVI', '29_NDVI', '30_NDVI', '31_NDVI']
inds = range(samples_datesets.size().getInfo())

In [11]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,0_NDVI,1_NDVI,2_NDVI,3_NDVI,4_NDVI,5_NDVI,6_NDVI,7_NDVI,8_NDVI,...,22_NDVI,23_NDVI,24_NDVI,25_NDVI,26_NDVI,27_NDVI,28_NDVI,29_NDVI,30_NDVI,31_NDVI
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(33),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
id,867.000000,868.000000,869.000000,870.000000,871.000000,872.000000,873.000000,940.000000,941.000000,942.000000,...,857.000000,858.000000,859.000000,860.000000,861.000000,862.000000,863.000000,864.000000,865.000000,866.000000
0_NDVI,0.411538,0.179964,0.200252,0.270593,0.226139,0.225331,0.215732,0.262540,0.344981,0.448837,...,0.468238,0.389246,0.516827,0.382436,0.366658,0.545799,0.558570,0.606455,0.580713,0.631876
1_NDVI,0.412755,0.189022,0.205667,0.272887,0.231356,0.230867,0.221366,0.272566,0.352706,0.459363,...,0.475961,0.396134,0.523212,0.384984,0.378619,0.550495,0.563061,0.613010,0.591083,0.633543
2_NDVI,0.415301,0.199448,0.211943,0.276024,0.237301,0.237124,0.228040,0.283434,0.361323,0.470559,...,0.484216,0.403943,0.529699,0.388431,0.391911,0.555419,0.567096,0.619439,0.601184,0.635062
3_NDVI,0.419160,0.211165,0.219037,0.279980,0.243934,0.244057,0.235704,0.295062,0.370761,0.482347,...,0.492945,0.412614,0.536249,0.392754,0.406435,0.560539,0.570657,0.625697,0.610945,0.636425


In [14]:
df.to_csv('sample_NDVI_2022_1000.csv',encoding='utf-8')